In [1]:
import requests
from bs4 import BeautifulSoup
import lxml.html
from selenium import webdriver
import time
import pandas as pd
import re
from tqdm import tqdm

In [2]:
reviews2 = pd.read_csv("top100&place.csv")

In [3]:
lq = pd.read_csv('bcstores.csv')

In [132]:
reviews2.head(20)

,mnf,name,style,alc,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews,rev_link,comb,mnf_comb
0,Four Winds Brewing Co.,nectarous,Wild Ale,5.50%,123,4.45,['look: 4.5 | smell: 4.75 | taste: 4.25 | feel...,"['4.36', '4.2', '5', '3.4', '3.9', '4.54', '4....",35,"[""Poured from the can into a teku glass. No vi...",https://www.beeradvocate.com/beer/profile/3218...,four winds - nectarous dry hopped sour,0
1,Driftwood Brewery,sartori harvest ipa,IPA - American,7.00%,117,4.40,['look: 4.25 | smell: 4.25 | taste: 4.25 | fee...,"['4.28', '4.3', '4.12', '4.24', '4.41', '4.32'...",42,"[""I've heard great things...now I've tasted th...",https://www.beeradvocate.com/beer/profile/1885...,NaN,-1
2,Driftwood Brewery,fat tug ipa,IPA - American,7.00%,718,4.27,['look: 3.75 | smell: 4.25 | taste: 4 | feel: ...,"['4.05', '4.31', '4.81', '4.17', '3.94', '3.88...",184,['Superb lacing & pours golden pale in the gla...,https://www.beeradvocate.com/beer/profile/1885...,driftwood brewery - fat tug ipa,0
3,Central City Brewers + Distillers,red racer ipa (india pale ale),IPA - American,6.50%,1136,4.25,['look: 4 | smell: 3 | taste: 4 | feel: 3 | ov...,"['3.46', '4.21', '3.86', '4.75', '4.5', '3.71'...",499,['Look: Clear dark copper color. Pours a light...,https://www.beeradvocate.com/beer/profile/1032...,NaN,-1
4,Driftwood Brewery,singularity,Stout - Russian Imperial,10.60%,180,4.28,['look: 4.25 | smell: 4.25 | taste: 4.5 | feel...,"['4.43', '4.87', '3.2', '4.1', '3.83', '4.21',...",70,"[""2oz bottle (2012, label says 14.0% ABV as op...",https://www.beeradvocate.com/beer/profile/1885...,NaN,-1
5,Driftwood Brewery,raised by wolves ipa,IPA - New England,7.00%,79,4.32,['look: 5 | smell: 4.75 | taste: 4.5 | feel: 5...,"['4.69', '3', '4.6', '4.17', '4.24', '4.46', '...",33,['Smells like one of those bags of dried fruit...,https://www.beeradvocate.com/beer/profile/1885...,driftwood brewery - raised by wolves ipa tall ...,0
6,Driftwood Brewery,naughty hildegard esb,Bitter - English Extra Special / Strong Bitter...,6.50%,140,4.25,['look: 4 | smell: 3 | taste: 3 | feel: 4 | ov...,"['3.16', '4.21', '4', '4.29', '4.1', '3.61', '...",47,"[""Clear dark amber color. The head was a light...",https://www.beeradvocate.com/beer/profile/1885...,NaN,-1
7,Four Winds Brewing Co.,juxtapose wild ipa,IPA - American,6.50%,133,4.20,['look: 4.25 | smell: 4.25 | taste: 4.5 | feel...,"['4.4', '4.19', '3.99', '2.6', '4.13', '4.13',...",41,"[""This is new to me. Poured from a 1 pint can,...",https://www.beeradvocate.com/beer/profile/3218...,four winds - juxtapose wild ipa,0
8,Central City Brewers + Distillers,red racer imperial ipa,IPA - Imperial,9.00%,190,4.17,['look: 4.25 | smell: 4.25 | taste: 4.25 | fee...,"['4.23', '3.26', '3.08', '4.15', '3.86', '3.4'...",66,"[""Appearance: Clear medium copper, decent carb...",https://www.beeradvocate.com/beer/profile/1032...,NaN,-1
9,Driftwood Brewery,latus,Sour - Flanders Red Ale,7.50%,53,4.30,"['SebD from Canada', 'look: 4 | smell: 4.25 | ...","['4.75', '4.47', '3.89', '4.21', '3.71', '4.46...",18,['Big thanks to FondueVoodoo for helping me ro...,https://www.beeradvocate.com/beer/profile/1885...,NaN,-1


In [133]:
reviews2.columns

Index(['mnf', 'name', 'style', 'alc', 'ratings', 'avg_rating', 'detail_rating',
       'avg_per_user', 'number_of_reviews', 'text_reviews', 'rev_link', 'comb',
       'mnf_comb'],
      dtype='object')

In [130]:
reviews2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mnf                403 non-null    object 
 1   name               403 non-null    object 
 2   style              403 non-null    object 
 3   alc                403 non-null    object 
 4   ratings            403 non-null    int64  
 5   avg_rating         403 non-null    float64
 6   detail_rating      403 non-null    object 
 7   avg_per_user       403 non-null    object 
 8   number_of_reviews  403 non-null    int64  
 9   text_reviews       403 non-null    object 
 10  rev_link           403 non-null    object 
 11  comb               315 non-null    object 
 12  mnf_comb           403 non-null    int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 41.1+ KB


In [5]:
lq.name = lq.name.str.lower()

In [6]:
reviews2.name = reviews2.name.str.lower()

In [7]:
reviews2.name.index

RangeIndex(start=0, stop=403, step=1)

In [30]:
shared_name = reviews2.name.apply(lambda x: lq[lq.name.str.lower().str.find(x).ge(0)].name).bfill(axis =1).iloc[:,0]
shared_name

0                 four winds - nectarous dry hopped sour
1                                                    NaN
2                        driftwood brewery - fat tug ipa
3                                                    NaN
4                                                    NaN
5      driftwood brewery - raised by wolves ipa tall ...
6                                                    NaN
7                        four winds - juxtapose wild ipa
8                                                    NaN
9                                                    NaN
10                                                   NaN
11                                                   NaN
12                                                   NaN
13                                                   NaN
14                                                   NaN
15                                                   NaN
16                                                   NaN
17                   yellow dog

In [36]:
dj = pd.DataFrame(list(zip(shared_name, reviews2.mnf, reviews2.name)), columns = ['lqbeer', 'mnfbeeravo','beeradvo'])

In [37]:
dj['lqmnf']= dj.lqbeer.astype(str).apply(lambda x: x.split("-")[0])

In [39]:
dj.mnfbeeravo = dj.mnfbeeravo.str.lower()

In [12]:
#dj.mnfbeeravo = dj.mnfbeeravo.str.replace("brewing","").str.replace("co.","").str.replace("BG","").str.replace(" ","")

In [34]:
#dj.lqmnf = dj.lqmnf.str.replace(" ","")

In [42]:
dj = dj[['lqbeer','beeradvo', 'lqmnf', 'mnfbeeravo']]

In [43]:
dj

,lqbeer,beeradvo,lqmnf,mnfbeeravo
0,four winds - nectarous dry hopped sour,nectarous,four winds,four winds brewing co.
1,NaN,sartori harvest ipa,nan,driftwood brewery
2,driftwood brewery - fat tug ipa,fat tug ipa,driftwood brewery,driftwood brewery
3,NaN,red racer ipa (india pale ale),nan,central city brewers + distillers
4,NaN,singularity,nan,driftwood brewery
5,driftwood brewery - raised by wolves ipa tall ...,raised by wolves ipa,driftwood brewery,driftwood brewery
6,NaN,naughty hildegard esb,nan,driftwood brewery
7,four winds - juxtapose wild ipa,juxtapose wild ipa,four winds,four winds brewing co.
8,NaN,red racer imperial ipa,nan,central city brewers + distillers
9,NaN,latus,nan,driftwood brewery


In [52]:
dj.lqmnf =dj.lqmnf.apply(lambda x: x[:-1] if x[-1]==" " else x)

In [112]:
dj.loc[100:103,'lqmnf'] = '33 acres'

In [113]:
res =[]
for i, j in zip(dj.mnfbeeravo, dj.lqmnf):
    print(i,j)
    m = i.find(j)
    print(m)
    res.append(m)

four winds brewing co. four winds
0
driftwood brewery nan
-1
driftwood brewery driftwood brewery
0
central city brewers + distillers nan
-1
driftwood brewery nan
-1
driftwood brewery driftwood brewery
0
driftwood brewery nan
-1
four winds brewing co. four winds
0
central city brewers + distillers nan
-1
driftwood brewery nan
-1
superflux nan
-1
driftwood brewery nan
-1
central city brewers + distillers nan
-1
parallel 49 brewing company nan
-1
steamworks brewing company nan
-1
four winds brewing co. nan
-1
driftwood brewery nan
-1
yellow dog brewing yellow dog
0
crannóg ales nan
-1
howe sound inn & brewing company nan
-1
driftwood brewery nan
-1
four winds brewing co. nan
-1
four winds brewing co. nan
-1
four winds brewing co. nan
-1
superflux nan
-1
lighthouse brewing company nan
-1
four winds brewing co. nan
-1
lighthouse brewing company lighthouse
0
boombox brewing company nan
-1
superflux nan
-1
phillips brewing company nan
-1
phillips brewing company nan
-1
storm brewing nan
-1
ye

In [114]:
dj['mnfcomp'] =res

In [115]:
pd.set_option('display.max_rows', 1000)


In [116]:
dj[dj.mnfcomp==0]

,lqbeer,beeradvo,lqmnf,mnfbeeravo,mnfcomp
0,four winds - nectarous dry hopped sour,nectarous,four winds,four winds brewing co.,0
2,driftwood brewery - fat tug ipa,fat tug ipa,driftwood brewery,driftwood brewery,0
5,driftwood brewery - raised by wolves ipa tall ...,raised by wolves ipa,driftwood brewery,driftwood brewery,0
7,four winds - juxtapose wild ipa,juxtapose wild ipa,four winds,four winds brewing co.,0
17,yellow dog - play dead ipa tall can,play dead ipa,yellow dog,yellow dog brewing,0
27,lighthouse - shipwreck ipa can,shipwreck ipa,lighthouse,lighthouse brewing company,0
34,yellow dog brewing - high 5 hazy ipa can,high 5 hazy ipa,yellow dog brewing,yellow dog brewing,0
54,russell brewing - a wee angry scotch ale,angry scotch ale,russell brewing,russell brewing company,0
64,parallel 49 - schadenfreude pumpkin oktoberfes...,schadenfreude,parallel 49,parallel 49 brewing company,0
66,vancouver island - hermannator ice bock tall can,hermannator,vancouver island,vancouver island brewing,0


In [117]:
dj[(dj.mnfcomp!=0) & (dj.lqmnf != 'nan')]

,lqbeer,beeradvo,lqmnf,mnfbeeravo,mnfcomp
73,fernie - project 9 pilsner can,pilsner,fernie,four winds brewing co.,-1
104,fernie brewing - the real peel grapefruit ipa ...,grapefruit ipa,fernie brewing,4 mile brewing co.,-1
105,alexander keith's - india pale ale can,india pale ale,alexander keith's,4 mile brewing co.,-1
106,33 acres of ocean pale ale,pale ale,33 acres of ocean pale ale,4 mile brewing co.,-1
107,new belgium - fat tire amber ale,amber ale,new belgium,andina brewing company,-1
108,tofino brewing - tuff session ale,session ale,tofino brewing,andina brewing company,-1
109,pacific western - canterbury dark mild can,dark mild,pacific western,backroads brewing company,-1
110,russell brewing - blood alley esb tall can,esb,russell brewing,backroads brewing company,-1
111,lighthouse brewing company - cloudburst hefewe...,hefeweizen,lighthouse brewing company,backroads brewing company,-1
112,mt begbie - high country kolsch can,kolsch,mt begbie,backroads brewing company,-1


In [119]:
shared_name

0                 four winds - nectarous dry hopped sour
1                                                    NaN
2                        driftwood brewery - fat tug ipa
3                                                    NaN
4                                                    NaN
5      driftwood brewery - raised by wolves ipa tall ...
6                                                    NaN
7                        four winds - juxtapose wild ipa
8                                                    NaN
9                                                    NaN
10                                                   NaN
11                                                   NaN
12                                                   NaN
13                                                   NaN
14                                                   NaN
15                                                   NaN
16                                                   NaN
17                   yellow dog

In [120]:
reviews2['comb'] = shared_name

In [121]:
reviews2['mnf_comb'] = dj.mnfcomp

In [122]:
reviews2_to_merge = reviews2[reviews2.mnf_comb==0]

In [123]:
len(reviews2_to_merge.comb)

106

In [124]:
df =pd.merge(lq, reviews2_to_merge, how ='left', left_on = lq.name, right_on = reviews2_to_merge.comb)

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515 entries, 0 to 514
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   key_0              515 non-null    object 
 1   name_x             515 non-null    object 
 2   country            515 non-null    object 
 3   price              515 non-null    object 
 4   sku_link           515 non-null    object 
 5   style_x            515 non-null    object 
 6   alc_x              515 non-null    object 
 7   beer_type          515 non-null    object 
 8   desc               515 non-null    object 
 9   mnf                112 non-null    object 
 10  name_y             112 non-null    object 
 11  style_y            112 non-null    object 
 12  alc_y              112 non-null    object 
 13  ratings            112 non-null    float64
 14  avg_rating         112 non-null    float64
 15  detail_rating      112 non-null    object 
 16  avg_per_user       112 non

In [131]:
df.columns

Index(['key_0', 'name_x', 'country', 'price', 'sku_link', 'style_x', 'alc_x',
       'beer_type', 'desc', 'mnf', 'name_y', 'style_y', 'alc_y', 'ratings',
       'avg_rating', 'detail_rating', 'avg_per_user', 'number_of_reviews',
       'text_reviews', 'rev_link', 'comb', 'mnf_comb'],
      dtype='object')

In [128]:
df[df.duplicated(subset=['name_y'])]

,key_0,name_x,country,price,sku_link,style_x,alc_x,beer_type,desc,mnf,...,alc_y,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews,rev_link,comb,mnf_comb
5,alexander keith's - india pale ale can,alexander keith's - india pale ale can,Canada,$25.99,/product/588681,India Pale Ale,5%,Domestic Beer - National Domestic Beer,"Brewed in Halifax since 1820, India Pale Ale i...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,alexander keith's - india pale ale can,alexander keith's - india pale ale can,Canada,$14.99,/product/687251,India Pale Ale,5%,Domestic Beer - National Domestic Beer,"Brewed in Halifax since 1820, India Pale Ale i...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,alhambra reserva 1925,alhambra reserva 1925,Spain,$13.29,/product/638759,Pilsner,6.4%,Import Beer,This award-winning lager is brewed over 35 day...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,asahi super dry,asahi super dry,Thailand,$14.79,/product/439950,Pilsner,5%,Import Beer,no info,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,asahi super dry big can,asahi super dry big can,Japan,$14.79,/product/541201,Pilsner,5%,Import Beer,no info,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,balderdash brewing - sneaky weasel craft lager,balderdash brewing - sneaky weasel craft lager,Canada,$18.49,/product/20555,Lager,5.6%,Domestic Beer - BC Craft Beer,"The Weasels’ Den 15-pack is locally brewed, su...",Balderdash Brewing Co.,...,5.6%,4.0,3.45,"['Apr 08, 2020']",['3.6'],1.0,"[""Clear yellow-gold color. Tiny head that quic...",/beer/profile/52950/350203/,balderdash brewing - sneaky weasel craft lager,0.0
12,balderdash brewing - sneaky weasel craft lager...,balderdash brewing - sneaky weasel craft lager...,Canada,$7.79,/product/994145,Lager,5.6%,Domestic Beer - BC Craft Beer,This mildy hopped craft lager delivers a bold ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,balderdash brewing - sneaky weasel craft lager...,balderdash brewing - sneaky weasel craft lager...,Canada,$1.99,/product/198948,Lager,5.6%,Domestic Beer - BC Craft Beer,Now in a tall can! This mildly hopped craft l...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,baltika 7,baltika 7,Russia,$2.49,/product/176909,Lager,5.4%,Import Beer,no info,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,bards tale - gold gluten free lager can,bards tale - gold gluten free lager can,USA,$15.29,/product/193590,Gluten-free,4.7%,Import Beer,100% gluten-free lager brewed with malted sorg...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
df.name_y

0                   33 acres of life
1                33 acres of nirvana
2                  33 acres of ocean
3               33 acres of sunshine
4                                NaN
5                                NaN
6                                NaN
7                                NaN
8                                NaN
9                                NaN
10                     sneaky weasel
11                     sneaky weasel
12                               NaN
13                               NaN
14                               NaN
15                               NaN
16                     52 foot stout
17                               NaN
18                               NaN
19                               NaN
20                               NaN
21                               NaN
22                               NaN
23                               NaN
24                               NaN
25                               NaN
26                   traditional ale
2